## Importing neccesary libraries
- We are using transfer learning here


In [1]:
import torch.nn as nn 
import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import torchvision.transforms as transforms
from PIL import Image
import torch.nn.functional as F
from tqdm import tqdm

## Using MobileNetV2 Model

In [2]:
model = models.mobilenet_v2(pretrained=True)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


100%|██████████| 13.6M/13.6M [00:00<00:00, 203MB/s]


In [3]:
model

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

## Freezing the params

In [4]:
for param in model.features.parameters():
    param.requires_grad = False

## Creating Custom Dataset

In [5]:
class CustomDataset(Dataset):
    def __init__(self, path_dir, transform=None):
        self.path_dir = Path(path_dir)
        self.transform = transform
        self.img_paths = []
        self.img_labels = []
        self.class_names = sorted([d.name for d in self.path_dir.iterdir() if d.is_dir()])
        self.classes = self.class_names
        for label, class_names in enumerate(self.classes):
            class_dir = self.path_dir/class_names
            for img_file in class_dir.iterdir():
                if img_file.suffix.lower() in {".png",".jpg",".jpeg"}:
                    self.img_paths.append(str(img_file))
                    self.img_labels.append(label)
        print(f"Images loaded {len(self.img_paths)} form {len(self.classes)} labels ")
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img_labels = self.img_labels[index]
        image = Image.open(self.img_paths[index]).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.img_labels[index]

## Creating Transformers

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(0.5),
    transforms.RandomVerticalFlip(0.5),
    transforms.RandomRotation(degrees=30),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

## Essential Things

In [7]:
Dir_path_train = "/kaggle/input/american-sign-language/ASL_Dataset/Train"
Dir_path_test = "/kaggle/input/american-sign-language/ASL_Dataset/Test"

train_Dataset = CustomDataset(Dir_path_train, train_transform)
test_Dataset = CustomDataset(Dir_path_test, val_transform)
Train_Loader = DataLoader(train_Dataset, batch_size=32, shuffle=True)
Test_Loader = DataLoader(test_Dataset, batch_size=32, shuffle=False)

model.classifier[1] = nn.Linear(1280, len(train_Dataset.classes))

Optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
criterion = nn.CrossEntropyLoss()

Images loaded 165670 form 28 labels 
Images loaded 112 form 28 labels 


## Evaluation Function

In [8]:
def evaluate(model, dataloader, device):
    model.eval()
    correct, total = 0.0, 0.0

    with torch.no_grad():
        for images, labels in dataloader:
            if images is None:
                continue
            images, labels = images.to(device), labels.to(device)
            preds = torch.argmax(model(images), dim = 1)
            correct += (preds==labels).sum().item()
            total += labels.size(0)
    accuracy = correct/total if total>0 else 0.0
    model.train()
    return accuracy

## Defining Training Loop

In [9]:
def train(model, train_loader, test_loader, device, epochs=10):
    model.train()
    for epoch in range(epochs):
        train_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{epochs}]", leave=False)
        for x, y in progress_bar:
            Optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            if x is None:
                continue
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            Optimizer.step()
            train_loss = train_loss + loss.item()
            progress_bar.set_postfix(loss=loss.item())

        val_accuracy = evaluate(model, test_loader, device)
        avg_train_loss = train_loss / len(train_loader)
        
        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} | Val Acc: {val_accuracy:.4f}")
    MODEL_PATH = "sign_language.pth"
    torch.save(model.state_dict(), MODEL_PATH)
    print(f"✅ Model saved to {MODEL_PATH}")
            

## Preparing Traning

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
train(model.to(device), Train_Loader, Test_Loader, device)

Epoch [1/10] | Train Loss: 0.8942 | Val Acc: 0.9196


Epoch [2/10] | Train Loss: 0.2399 | Val Acc: 0.9554


Epoch [3/10] | Train Loss: 0.1595 | Val Acc: 0.9911


Epoch [4/10] | Train Loss: 0.1289 | Val Acc: 0.9821


Epoch [5/10] | Train Loss: 0.1116 | Val Acc: 0.9821


Epoch [6/10] | Train Loss: 0.1013 | Val Acc: 0.9821


Epoch [7/10] | Train Loss: 0.0942 | Val Acc: 0.9821


Epoch [8/10] | Train Loss: 0.0894 | Val Acc: 0.9821


Epoch [9/10] | Train Loss: 0.0862 | Val Acc: 0.9911


Epoch [10/10] | Train Loss: 0.0820 | Val Acc: 0.9821
✅ Model saved to sign_language.pth
